In [2]:
import numpy as np
from sklearn import datasets
# import matplotlib.pyplot as plt

---
## Datenauszug: Trainingsdaten und Kontrolldaten

In [3]:
# we are going to classify a popular breast cancer dataset
data, target = datasets.load_breast_cancer(return_X_y=True)

---
## Shape der Daten ausgeben

In [4]:
data.shape
# data is a matrix of shape (569, 30)

(569, 30)

---
## Sneakpeak der Target-Daten

In [5]:
# target is a binary vector of size (569,) in which each entry is the label of
# each case, either malignant (0) or benign (1).
target[:20]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

---
## Bearbeite Daten: 30 -> 2 Features

In [6]:
# for simplicity we cut the number of features from 30 to just two features
data = data[:, :2]

---
## Aufgabe 1.1

Es wird ein Klassifizierer nach dem Vorbild aus der Übung geschrieben, welcher basierend auf zwei Datenpunkten entschiedet, ob eine Person Krebs hat oder nicht. Diese Vorhersage wird daraufhin durch eine Accuracy-Funktion qualitativ bewertet.

In [48]:
# Meine Test-Weights:
test_weights = [-10,3,100]



# Funktion, welche zu einem gegebenen Datensatz ein Array der Dimension len(datensatz)x1 berechnet.
# Das Arrays ist eine Vorhersage/Zuordnung des Klassifizierers der gegebene Daten.
def classifier_scores(data, weights):

    classified = [0] * len(data)

    for i in range(0,len(data)):
        classified[i] = weights[0]*data[i,0] + weights[1]*data[i,1] + weights[2]

    return classified



# Diese Funktion gibt an, wie genau die Scores bzgl. der gegebenen Target-Daten sind:
def accuracy(scores, targets, weights):

    # Zähler für korrekte Vorhersagen:
    correct = 0

    # Schreibe scores so, dass ein Vgl. mit den Target-Daten einfacher ist:
    for i in range(0, len(scores)):
        if scores[i] <= 0.5:
            scores[i] = 0
        else:
            scores[i] = 1

    # Vgl. zwischen Vorhersagen und Target-Daten:
    for i in range(0,len(scores)):
        if scores[i] == targets[i]:
            correct += 1

    # Konsolenausgabe:
    print("Übereinstimmung der Prediction in %: " + str(correct/596))
    print("\nDie Geradengleichung lautet: " + str(round(weights[0], 3)) + "x+" + str(round(weights[1], 3)) + "y+" + str(round(weights[2], 3)))

    # Gebe Ergebnis aus:
    return correct/596

# Resultate aus meinen Test-Weights
test_scores   = classifier_scores(data, test_weights)
test_accuracy = accuracy(test_scores, target, test_weights)

Übereinstimmung der Prediction in %: 0.7701342281879194

Die Geradengleichung lautet: -10x+3y+100


---
## Aufgabe 1.2

Die Lossfunktion ist gegeben durch:

$$l=(t-tp)^2=(t-w_1f_1-w_2f_2-w_3)^2$$

In Python ist f1 = data[i][0] und f2 = data[i][1]. Die Gewcihte w_i sind gegeben durch w_1 = w[0], w_2 = w[1], und w_3 = w[2]: 

$$l = \frac{1}{N} \sum_{i=0}^{N-1} (t[i] - w[0]d[i][0] - w[0]d[i][1] - w[2])^2$$
$$l = \frac{1}{N} \sum_{i=0}^{N-1} (t_i -w_1f_1 - w_2f_2 - w_3)^2$$

Um gradient descent anzuwenden, müssen zunächst alle Ableitungen berechnet werden:

1) Ableitung nach w_1

$$\frac{\partial l}{\partial w[0]} = \frac{1}{N} \sum_{i=0}^{N-1} -2d[i][0](t[i] - w[0]d[i][0] - w[0]d[i][1] - w[2])$$
$$\frac{\partial l}{\partial w_1} = \frac{1}{N} \sum_{i=0}^{N-1} -2f_1(t_i - -w_1f_1 - w_2f_2 - w_3)$$

2) Ableitung nach w_2

$$\frac{\partial l}{\partial w[1]} = \frac{1}{N} \sum_{i=0}^{N-1} - 2d[i][1](t[i] - w[0]d[i][0] - w[0]d[i][1] - w[2])$$
$$\frac{\partial l}{\partial w_2} = \frac{1}{N} \sum_{i=0}^{N-1} - 2f_2(t_i - -w_1f_1 - w_2f_2 - w_3)$$

3) Ableitung nach w_3 

$$\frac{\partial l}{\partial w[2]} = \frac{1}{N} \sum_{i=0}^{N-1} -2(t[i] - w[0]d[i][0] - w[0]d[i][1] - w[2])$$
$$\frac{\partial l}{\partial w_3} = \frac{1}{N} \sum_{i=0}^{N-1} - 2(t_i - -w_1f_1 - w_2f_2 - w_3)$$

Wir rechnen mit 1000 Epochen.

Wir rechnen mit einer Learning-Rate von 0.001

In [49]:
# Implementation gradient descent für unser Beispiel
def gradient_descent(d, t):

    # Lege Epochen und Learning-Rate fest:
    epochs        = 10000
    learning_rate = 0.01

    # Startwerte für die Parameter sowie deren Änderung per default = 0 bei gradient descent:
    w  = [0, 0, 0]
    dw = [0, 0, 0]

    # Um unnötige Aufrufe der len() Funktionzu vermeiden:
    N = len(d)

    for i in range(0, epochs):
        
        # Berechne den Gradienten => Sprich für jede Zeile der d-Matrix die dazugehörigen partiellen Ableitungen
        for i in range(0, N):

            # Um redundante Berechnungen zu vermeiden:
            cond = w[0]*d[i][0] + w[1]*d[i][1] + w[2]
            mult = 0
            if cond < 0.5:
                mult = t[i] 
            else:
                mult = t[i] - 1
            
            # dl_d(w0):
            dw[0] += -2*d[i][0] * mult/N
            # dl_d(w1):
            dw[1] += -2*d[i][1] * mult/N
            # dl_d(w2):
            dw[2] += -2 * mult/N

        # Update die Parameter gemäß der Learning-Rate am Ende einer Epoche:
        for i in range(0, 3):
            w[i] -= learning_rate * dw[i]

    # Gebe die gefundenen Parameter aus:
    return w



# Anwendung
results = gradient_descent(data, target)
print(results)

# Auswertung
scores = classifier_scores(data, results)
acc    = accuracy(scores, target, results)

[-35.87346098404238, -10.120059050581045, 719.0773286466376]
Übereinstimmung der Prediction in %: 0.8523489932885906

Die Geradengleichung lautet: -35.873x+-10.12y+719.077


---
## GitHub

Nachfolgend der GitHub Link zum Repo für diesen Praktikumsversuch. Dort sind folgende Unterlagen hinterlegt:

- JULIA Implementation der Lösungen zu den Aufgaben
- Jupyter Notebook Implementation der Lösungen zu den Aufgaben

https://github.com/felix12123/CP-Machine_Learning